(a) What is ImageDataGenerator and what does it do? What are the “types” of train_datagen and valid_datagen and how are they used in the code?

(b) Explain the architecture of the CNN model used in the code, run it and report the results.

(c) Modify the architecture to improve the default model that is in the code. Explain what steps you took to try to improve the accuracy of the default model. (If none of the steps improved the model present the results of the “best” attempt to improve the model, and explain the changes that you tried).

In [25]:
from __future__ import print_function
import tensorflow as tf
from tensorflow import keras
import keras
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
from PIL import Image
import sys
import cv2
import matplotlib.pyplot as plt

### What is ImageDataGenerator and what does it do? What are the “types” of train_datagen and valid_datagen and how are they used in the code?

ImageDataGenerator is a preprocessing tool to augment the image data. Augmenting data is to create/generate data similar to what is already in our dataset with some differences that can be determined by us. This helps to increase the number of training data that we have to train a more accurate model.

What is happening with the train_datagen and valid_datagen is that we are rescaling the image pixels [0,255] to [0,1]. This helps the process of training and predicting faster especially with more data, and additionally and probably most importantly, it will prevent the models and the weights of the cnn model to be skewed towards higher pixels. Normalizes the data in a way!

In [38]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
valid_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
os.chdir('/Users/alisdghnia/Desktop/')

In [5]:
train_generator = train_datagen.flow_from_directory(
    directory="FRUITS/Training32",
    target_size=(32, 32),
    color_mode="rgb",
    batch_size=5,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 12642 images belonging to 25 classes.


In [6]:
valid_generator = valid_datagen.flow_from_directory(
    directory="FRUITS/Test32",
    target_size=(32, 32),
    color_mode="rgb",
    batch_size=5,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 4232 images belonging to 25 classes.


In [35]:
train_generator.image_shape

(32, 32, 3)

### Explain the architecture of the CNN model used in the code, run it and report the results.

Mainly with the Conv2D layers, we assign the number of filters, where in each epoch they are trained to detect specific features in the images. The training of the filters is done through minimizing the loss. Then there is the Kernel Size which is (3,3) and is the size of the convolution window. The kernel size is what the filter sizes are and what the images are 'processed' through for the output layer to come out. if there is padding of 'same' or 1 then the output layer will be the same size as input layer which in this case it is (32,32) otherwise if the padding of 1 is not present then the output is going to lose one line from all 4 sides in this case (different filters and kernel sizes will change the output shape).
Each time the activation function ReLU is called that turns negative values received in the layer after each convolution to 0 and keeps the positive values. It is a faster process than most other activation functions. Only the positive values staying will emphesize the nodes that result into positive values therefore making the process faster; however, this could lead to nodes losing functionality as the model trains epoch after epoch.
Therefore, the use of dropout will help to prevent the nodes from becoming redondent in their work and make sure all nodes matter in the training and testing.
Maxpooling then is added to the sequence to downsample the input by grabbing the maximum value from the (2,2) window of the pool size on the layer.

Repeated X2

Then the values are flattened to convert the data to a 1-d array in order to create a feature vector and basically use the feature vector to then dense it into the number of classification outputs we have at the end. (while also adding some activation and drop out in the middle).
And softmax activation is for the outputs of more than 2 categories.

Lastly, Adam optimizer with a learning rate of 0.0001 is used as the optimizer for the learning of the model and uses the adam algorithm to calculate the weights through backpropagation.

Loss function is categorical crossentropy which is a common loss function used for multi-class classification problems.


Output will be 25 indicating which of the 25 keys in the dictionary (which is all the categories of the items in the images) the image is showing!

In [7]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(32,32,3)))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(25))
model.add(Activation('softmax'))

model.summary()

# initiate adam optimizer
opt = tf.keras.optimizers.Adam(lr=0.0001)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 32)        0         
                                                        

2022-11-18 23:59:47.066031: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/alisdghnia/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [8]:
sys.modules['Image'] = Image

In [10]:
model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=30,
        validation_data=valid_generator,
        validation_steps=100)

Epoch 1/30
 12/100 [==>...........................] - ETA: 0s - loss: 0.2096 - accuracy: 0.9500

/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_23926/2210581391.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


100/100 [==============================] - 1s 11ms/step - loss: 0.2807 - accuracy: 0.9020 - val_loss: 0.3355 - val_accuracy: 0.8800
Epoch 2/30
100/100 [==============================] - 1s 11ms/step - loss: 0.3685 - accuracy: 0.8780 - val_loss: 0.3527 - val_accuracy: 0.8660
Epoch 3/30
100/100 [==============================] - 1s 10ms/step - loss: 0.3739 - accuracy: 0.8760 - val_loss: 0.3927 - val_accuracy: 0.8520
Epoch 4/30
100/100 [==============================] - 1s 10ms/step - loss: 0.2371 - accuracy: 0.9280 - val_loss: 0.2971 - val_accuracy: 0.8780
Epoch 5/30
100/100 [==============================] - 1s 10ms/step - loss: 0.2628 - accuracy: 0.9080 - val_loss: 0.2219 - val_accuracy: 0.9160
Epoch 6/30
100/100 [==============================] - 1s 10ms/step - loss: 0.2159 - accuracy: 0.9320 - val_loss: 0.3222 - val_accuracy: 0.9020
Epoch 7/30
100/100 [==============================] - 1s 11ms/step - loss: 0.2270 - accuracy: 0.9080 - val_loss: 0.3316 - val_accuracy: 0.8680
Epoch 8/30

In [ ]:
#Predict a single image
import cv2
from matplotlib import pyplot as plt

img = cv2.imread("FRUITS/Test32/Apple Braeburn/3_100.jpg")


#CV2 reads an image in BGR format. We need to convert it to RGB
b,g,r = cv2.split(img)       # get b,g,r
rgb_img = cv2.merge([r,g,b])     # switch it to rgb


plt.imshow(rgb_img)

In [ ]:
#We need to have the image as a tensor of rank 4
#Because we trained the model as (batch_size, height, width, channel)
#Here, batch_size will be one

#Also divide the image values by 255 to normalize

img_rank4 = np.expand_dims(rgb_img/255, axis=0)

model.predict(img_rank4)

In [ ]:
np.round(model.predict(img_rank4))

In [ ]:
# NOT SHOWN IN THE TUTORIAL VIDEO:
# We can predict the class directly using the following function:

# model.predict_classes(img_rank4)
predict_x=model.predict(img_rank4) 
classes_x=np.argmax(predict_x,axis=1)

In [19]:
# NOT SHOWN IN THE TUTORIAL VIDEO:
# We can retrieve the class labels from the train_generator:

label_map = (train_generator.class_indices)

label_map

{'Apple Braeburn': 0,
 'Apple Golden 1': 1,
 'Apple Golden 2': 2,
 'Apple Golden 3': 3,
 'Apple Granny Smith': 4,
 'Apple Red 1': 5,
 'Apple Red 2': 6,
 'Apple Red 3': 7,
 'Apple Red Delicious': 8,
 'Apple Red Yellow': 9,
 'Apricot': 10,
 'Avocado': 11,
 'Avocado ripe': 12,
 'Banana': 13,
 'Banana Red': 14,
 'Cactus fruit': 15,
 'Cantaloupe 1': 16,
 'Cantaloupe 2': 17,
 'Carambula': 18,
 'Cherry 1': 19,
 'Cherry 2': 20,
 'Cherry Rainier': 21,
 'Cherry Wax Black': 22,
 'Cherry Wax Red': 23,
 'Cherry Wax Yellow': 24}

In [23]:
# NOT SHOWN IN THE TUTORIAL VIDEO:
# We can retrieve the class label of the prediction:

# list(label_map.keys())[model.predict_classes(img_rank4)[0]]

list(label_map.keys())[classes_x[0]]

'Apple Braeburn'

### Modify the architecture to improve the default model that is in the code. Explain what steps you took to try to improve the accuracy of the default model. (If none of the steps improved the model present the results of the “best” attempt to improve the model, and explain the changes that you tried).

In [43]:
model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same', input_shape=(32,32,3)))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.05))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.1))

model.add(Flatten())

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(rate=0.15))

model.add(Dense(25))
model.add(Activation('softmax'))

model.summary()

# initiate adam optimizer
opt = tf.keras.optimizers.Adam(lr=0.0005)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 32, 32, 64)        1792      
                                                                 
 activation_18 (Activation)  (None, 32, 32, 64)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 30, 30, 64)        36928     
                                                                 
 activation_19 (Activation)  (None, 30, 30, 64)        0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 15, 15, 64)       0         
 2D)                                                             
                                                                 
 dropout_9 (Dropout)         (None, 15, 15, 64)        0         
                                                      

In [44]:
model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=30,
        validation_data=valid_generator,
        validation_steps=100)

Epoch 1/30


/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_23926/2210581391.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


100/100 [==============================] - 3s 25ms/step - loss: 2.7122 - accuracy: 0.1820 - val_loss: 1.8112 - val_accuracy: 0.3840
Epoch 2/30
100/100 [==============================] - 2s 23ms/step - loss: 1.5373 - accuracy: 0.4820 - val_loss: 1.3191 - val_accuracy: 0.6280
Epoch 3/30
100/100 [==============================] - 2s 25ms/step - loss: 0.9226 - accuracy: 0.6860 - val_loss: 1.1677 - val_accuracy: 0.5780
Epoch 4/30
100/100 [==============================] - 2s 24ms/step - loss: 0.6680 - accuracy: 0.7860 - val_loss: 0.7538 - val_accuracy: 0.7580
Epoch 5/30
100/100 [==============================] - 2s 23ms/step - loss: 0.5169 - accuracy: 0.8260 - val_loss: 0.8999 - val_accuracy: 0.6660
Epoch 6/30
100/100 [==============================] - 2s 24ms/step - loss: 0.4632 - accuracy: 0.8540 - val_loss: 0.8746 - val_accuracy: 0.7180
Epoch 7/30
100/100 [==============================] - 2s 24ms/step - loss: 0.4338 - accuracy: 0.8480 - val_loss: 0.5246 - val_accuracy: 0.8320
Epoch 8/30

### Interpretation

Originally in this notebook the first model was only trained with the epoch set at 5 which did not create a model that could predict very accurately. I changed the batch size in the beginning and increased the number of epoch to 30. Lowered the dropout and increased the number of filters. Basically I overfit the heck out of the model just to find a really high accuracy which ended with 96% accuracy! However, the Validation Loss and Accuracy could even be worked on more if we augment the training data more and also maybe make the architecture of the model more complex. But that would be pointless as this is already a great model to predict the fruits in the images!